# Importing Libraries

In [1]:
import numpy 
import pandas 
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import keras

Using TensorFlow backend.


# load dataset

In [3]:
dataframe = pandas.read_csv("housing.csv", delim_whitespace=True, header=None)
dataset = dataframe.values
X = dataset[:,0:13]
Y = dataset[:,13]
p = pandas.DataFrame(X)
p.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


# define base model

In [4]:
def baseline_model():
    model = Sequential()
    model.add(Dense(13, activation="relu", input_shape=(13,)))
    model.add(Dense(1, activation="relu"))
    
    model.compile(loss='mean_squared_error',
                 optimizer="adam")
    
    return model

In [5]:
seed = 7
numpy.random.seed(seed)

# evaluate model with standardized dataset

In [87]:
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)

In [88]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Results: -323.94 (400.26) MSE


# Evaluate model with standardized dataset

In [89]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: -27.90 (35.11) MSE


# Define larger model

In [94]:
def larger_model():
    model = Sequential()
    model.add(Dense(13, activation="relu", input_shape=(13,)))
    model.add(Dense(6, activation="relu"))
    model.add(Dense(1, activation="relu"))
    model.compile(loss="mean_squared_error",
                  optimizer="adam")
    
    return model

In [95]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -22.85 (29.51) MSE


# Define wider model

In [10]:
def wider_model():
    model = Sequential()
    model.add(Dense(20, activation="relu", input_shape=(13,)))
    model.add(Dense(1, activation="relu"))
    
    model.compile(loss="mean_squared_error",
                  optimizer="adam")
    
    return model

In [11]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

W0826 14:35:30.052276  1656 deprecation_wrapper.py:119] From C:\Games\envs\pro\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0826 14:35:30.549974  1656 deprecation_wrapper.py:119] From C:\Games\envs\pro\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0826 14:35:30.563639  1656 deprecation_wrapper.py:119] From C:\Games\envs\pro\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0826 14:35:30.862256  1656 deprecation_wrapper.py:119] From C:\Games\envs\pro\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0826 14:35:32.305592  1656 deprecation_wrapper.py:119] From C:\Games\envs\pro\lib\site-packages\keras\backe

Wider: -19.06 (21.08) MSE


# Model That OverFits

In [12]:
def overfit_model():
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim=13, kernel_initializer='normal', activation='relu'))
    model.add(Dense(13, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(3, activation='relu'))
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [13]:
np.random.seed(seed)
estimators = []
estimators.append(('standardize' , StandardScaler()))
estimators.append(('mlp' , KerasRegressor(build_fn = overfit_model , epochs = 50 , batch_size = 5 , verbose = 0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits = 10 , random_state = seed)
results = cross_val_score(pipeline , X , Y , cv = kfold)
print("Results: %2f(%2f)MSE"%(abs(results.mean()),results.std()))

Results: 20.827752(22.461489)MSE


# Model with some tuning

In [14]:
def tune_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1, activation='relu'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [15]:
np.random.seed(seed)
estimators = []
estimators.append(('standardize' , StandardScaler()))
estimators.append(('mlp' , KerasRegressor(build_fn = tune_model , epochs = 50 , batch_size = 5 , verbose = 0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits = 10 , random_state = seed)
results = cross_val_score(pipeline , X , Y , cv = kfold)
print("Results: %2f(%2f)MSE"%(abs(results.mean()),results.std()))

Results: 22.717213(24.677621)MSE


# Rewriting the code using Keras Functional API

In [19]:
from keras.layers import Input, Dense
from keras.models import Model

def functional_model():
    inputs = Input(shape = (13,))
    z1 = Dense(13, kernel_initializer="normal", activation='relu')(inputs)
    z2 = Dense(6, kernel_initializer="normal", activation='relu')(z1)
    outputs = Dense(1)(z2)
    model = Model(inputs,outputs)
    
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    return model

In [ ]:
np.random.seed(seed)
estimators = []
estimators.append(('standardize' , StandardScaler()))
estimators.append(('mlp' , KerasRegressor(build_fn = functional_model , epochs = 50 , batch_size = 5 , verbose = 0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits = 10 , random_state = seed)
results = cross_val_score(pipeline , X , Y , cv = kfold)
print("Results: %2f(%2f)MSE"%(abs(results.mean()),results.std()))

# Model Subclassing

In [6]:
import tensorflow as tf
class SubclassModel(tf.keras.Model):
    def __init__(self):
        super(SubclassModel,self).__init__()
        self.dense1= tf.keras.layers.Dense(13,activation=tf.nn.relu)
        self.dense2=tf.keras.layers.Dense(6,activation=tf.nn.relu)
        self.dense3= tf.keras.layers.Dense(1)
    def call(self,inputs):
        x=self.dense1(inputs)
        x=self.dense2(x)
        return self.dense3(x)
def finalModel():
    model=SubclassModel()
    model.compile(optimizer='adam', loss='mse' ,metrics=['mae'])
    return model

In [8]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize' , StandardScaler()))
estimators.append(('mlp' , KerasRegressor(build_fn = finalModel , epochs = 50 , batch_size = 5 , verbose = 0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits = 10 , random_state = seed)
results = cross_val_score(pipeline , X , Y , cv = kfold)
print("Results: %2f(%2f)MSE"%(abs(results.mean()),results.std()))

W0826 15:51:08.038288  1236 deprecation.py:506] From C:\Games\envs\pro\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Results: 24.834916(28.549035)MSE


# Model without SciKit Learn

In [10]:
dataset = dataframe.values

In [11]:
complete_data = dataset.copy()
data = complete_data[:,0:13]
labels = complete_data[:,13]
train_data = data[:404]
train_labels = labels[:404]
test_data = data[404:]
test_labels = labels[404:]

In [12]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

In [13]:
from keras import models
from keras import layers
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [14]:
import numpy as np
k=4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

In [15]:
model = build_model()
model.fit(train_data, train_labels,
epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_labels)
print(test_mse_score)

W0826 17:06:52.905260  1236 deprecation_wrapper.py:119] From C:\Games\envs\pro\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0826 17:06:52.912092  1236 deprecation_wrapper.py:119] From C:\Games\envs\pro\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0826 17:06:53.001872  1236 deprecation_wrapper.py:119] From C:\Games\envs\pro\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0826 17:06:53.522021  1236 deprecation_wrapper.py:119] From C:\Games\envs\pro\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0826 17:06:55.063920  1236 deprecation_wrapper.py:119] From C:\Games\envs\pro\lib\site-packages\keras\backe

102/102 [==============================] - 1s 10ms/step
34.06661822749119
